In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import pickle
from imp import reload
from os.path import join

import numpy as np
from tqdm import tqdm

from data_loader import DataLoader
import medim
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [2]:
raw_path = '/home/mount/neuro-t01-hdd/Brats2017/data/raw/'
processed_path = '/mount/export/Brats2017/data/processed/'

data_loader = DataLoader(raw_path)
patients = data_loader.patients

In [24]:
def encode_segm(s):
    r = np.zeros((3, *s.shape), dtype=bool)
    r[0] = s > 0
    r[1] = (s == 1) | (s == 4)
    r[2] = (s == 4)
    return r

In [32]:
import pandas as pd
from collections import namedtuple

In [26]:
def preprocess():
    for patient in tqdm(patients):
        mscan = data_loader.load_mscan(patient)
        segmentation = data_loader.load_segmentation(patient)

        # Extract part with the brain
        mask = np.any(mscan, axis=0)
        mscan, segmentation = medim.bb.extract(mscan, segmentation, mask=mask)

        mscan = medim.prep.normalize_mscan(
            mscan, mean=False)
        msegm = encode_segm(segmentation)

        filename = join(processed_path, patient)
        np.save(filename+'_mscan', mscan)
        np.save(filename+'_msegm', msegm)

In [27]:
%lprun -f preprocess preprocess()

100%|██████████| 285/285 [03:44<00:00,  1.28s/it]
